In [1]:
import wave
import os
import io
import pathlib
from pathlib import Path
from scipy.io import wavfile
import sounddevice as sd
import time
import librosa as lb
import librosa.display as dsp

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from os import path, listdir
import pandas as pd
import sounddevice as sd

In [3]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [4]:
import sklearn.model_selection 
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [5]:
directory_train = r'train/'

voice_paths_train = []
class_labels_train = []

for class_label, class_dir in enumerate(listdir(directory_train)):
    voice_paths_train += [path.join(directory_train,class_dir,f) for f in listdir(path.join(directory_train,class_dir))]
    class_labels_train += [class_label for _ in listdir(path.join(directory_train,class_dir))]

voice_paths_train_df = pd.DataFrame(voice_paths_train, columns=['voice path'])
voice_paths_train_df['class labels'] = class_labels_train

In [6]:
directory_eval = r'eval/'

voice_paths_eval = []
class_labels_eval = []

for class_label, class_dir in enumerate(listdir(directory_eval)):
    voice_paths_eval += [path.join(directory_eval,class_dir,f) for f in listdir(path.join(directory_eval,class_dir))]
    class_labels_eval += [class_label for _ in listdir(path.join(directory_eval,class_dir))]

voice_paths_eval_df = pd.DataFrame(voice_paths_eval, columns=['voice path'])
voice_paths_eval_df['class labels'] = class_labels_eval

In [7]:
voice_paths_eval_df.dtypes

voice path      object
class labels     int64
dtype: object

In [8]:
def feature_ext(file_path):
    temp_sr, temp_data = wavfile.read(file_path)
    temp_data = temp_data.astype(float)
    temp_mfcc_sample = lb.feature.mfcc(y = temp_data, sr = temp_sr, n_mfcc = 32)
    if(temp_mfcc_sample.shape[1] != 32):
        temp_shell = np.zeros((32,32))
        temp_shell[0:temp_mfcc_sample.shape[0], 0:temp_mfcc_sample.shape[1]] += temp_mfcc_sample
        return temp_shell.reshape(temp_shell.shape[0],temp_shell.shape[1],1)
    return temp_mfcc_sample.reshape(temp_mfcc_sample.shape[0],temp_mfcc_sample.shape[1],1)

In [9]:
mfcc_samples_train = []

for paths in voice_paths_train_df['voice path']:
    data = feature_ext(paths)
    mfcc_samples_train.append(data)
voice_paths_train_df['mfcc sample'] = mfcc_samples_train
voice_paths_train_df.head()

,voice path,class labels,mfcc sample
0,train/bed\e0315cf6_nohash_0.wav,0,"[[[314.0396677261368], [315.06702121935933], [..."
1,train/bed\e1469561_nohash_0.wav,0,"[[[516.3689188580837], [517.5797463207749], [5..."
2,train/bed\e2362167_nohash_0.wav,0,"[[[609.269199607749], [594.9521730537415], [56..."
3,train/bed\e269bac0_nohash_0.wav,0,"[[[385.44315681122674], [385.60353792800225], ..."
4,train/bed\e269bac0_nohash_1.wav,0,"[[[374.02580469726905], [374.4279821151091], [..."


In [10]:
mfcc_samples_eval = []

for paths in voice_paths_eval_df['voice path']:
    data = feature_ext(paths)
    mfcc_samples_eval.append(data)
voice_paths_eval_df['mfcc sample'] = mfcc_samples_eval
voice_paths_eval_df.head()

,voice path,class labels,mfcc sample
0,eval/bed\00f0204f_nohash_0.wav,0,"[[[418.5343101561289], [419.3625039917852], [4..."
1,eval/bed\00f0204f_nohash_1.wav,0,"[[[418.9800915999155], [420.753602873334], [41..."
2,eval/bed\01b4757a_nohash_0.wav,0,"[[[815.7798857836873], [815.9507383141112], [8..."
3,eval/bed\01b4757a_nohash_1.wav,0,"[[[784.659111636587], [788.0930767510608], [78..."
4,eval/bed\01bcfc0c_nohash_0.wav,0,"[[[484.63262779527037], [484.86793869244326], ..."


In [11]:
mfcc_samples_train_np = np.asarray(mfcc_samples_train)
mfcc_samples_eval_np = np.asarray(mfcc_samples_eval)

In [12]:
train_output_df = voice_paths_train_df.pop('class labels')
eval_output_df = voice_paths_eval_df.pop('class labels')

In [13]:
mfcc_samples_train_output_np = np.array(train_output_df)
mfcc_samples_eval_output_np = np.array(eval_output_df)

In [91]:
model = models.Sequential()

In [92]:
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32,32,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (2, 2), activation='relu'))

In [93]:
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(30, activation='softmax'))

In [94]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_57 (Conv2D)           (None, 30, 30, 32)        320       
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 4, 4, 64)          36928     
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 1, 1, 64)        

In [95]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(mfcc_samples_train_np, mfcc_samples_train_output_np, epochs=200, 
                    validation_data=(mfcc_samples_eval_np, mfcc_samples_eval_output_np))

Epoch 1/200
183/183 [==============================] - 4s 21ms/step - loss: 3.3593 - accuracy: 0.0994 - val_loss: 3.3960 - val_accuracy: 0.0424
Epoch 2/200
183/183 [==============================] - 4s 19ms/step - loss: 3.2634 - accuracy: 0.2022 - val_loss: 3.3064 - val_accuracy: 0.1436
Epoch 3/200
183/183 [==============================] - 4s 19ms/step - loss: 3.1950 - accuracy: 0.2731 - val_loss: 3.3656 - val_accuracy: 0.0859
Epoch 4/200
183/183 [==============================] - 4s 20ms/step - loss: 3.1405 - accuracy: 0.3311 - val_loss: 3.3031 - val_accuracy: 0.1498
Epoch 5/200
183/183 [==============================] - 4s 20ms/step - loss: 3.0912 - accuracy: 0.3766 - val_loss: 3.1345 - val_accuracy: 0.3296
Epoch 6/200
183/183 [==============================] - 4s 20ms/step - loss: 3.0423 - accuracy: 0.4288 - val_loss: 3.1462 - val_accuracy: 0.3153
Epoch 7/200
183/183 [==============================] - 4s 20ms/step - loss: 3.0720 - accuracy: 0.3972 - val_loss: 3.3471 - val_accuracy:

183/183 [==============================] - 4s 20ms/step - loss: 2.6835 - accuracy: 0.7745 - val_loss: 2.8521 - val_accuracy: 0.6080
Epoch 58/200
183/183 [==============================] - 4s 21ms/step - loss: 2.6727 - accuracy: 0.7868 - val_loss: 2.8972 - val_accuracy: 0.5605
Epoch 59/200
183/183 [==============================] - 4s 21ms/step - loss: 2.6710 - accuracy: 0.7885 - val_loss: 2.8963 - val_accuracy: 0.5601
Epoch 60/200
183/183 [==============================] - 4s 20ms/step - loss: 2.6770 - accuracy: 0.7824 - val_loss: 2.8774 - val_accuracy: 0.5828
Epoch 61/200
183/183 [==============================] - 4s 21ms/step - loss: 2.6807 - accuracy: 0.7779 - val_loss: 2.9151 - val_accuracy: 0.5415
Epoch 62/200
183/183 [==============================] - 4s 20ms/step - loss: 2.6745 - accuracy: 0.7851 - val_loss: 2.9928 - val_accuracy: 0.4633
Epoch 63/200
183/183 [==============================] - 4s 21ms/step - loss: 2.6833 - accuracy: 0.7751 - val_loss: 2.9043 - val_accuracy: 0.555

183/183 [==============================] - 4s 21ms/step - loss: 2.6324 - accuracy: 0.8254 - val_loss: 2.8234 - val_accuracy: 0.6361
Epoch 114/200
183/183 [==============================] - 4s 21ms/step - loss: 2.6255 - accuracy: 0.8323 - val_loss: 2.8558 - val_accuracy: 0.6036
Epoch 115/200
183/183 [==============================] - 4s 21ms/step - loss: 2.6294 - accuracy: 0.8284 - val_loss: 2.8453 - val_accuracy: 0.6138
Epoch 116/200
183/183 [==============================] - 4s 21ms/step - loss: 2.8810 - accuracy: 0.5760 - val_loss: 2.9703 - val_accuracy: 0.4874
Epoch 117/200
183/183 [==============================] - 4s 21ms/step - loss: 2.7541 - accuracy: 0.7056 - val_loss: 3.3202 - val_accuracy: 0.1341
Epoch 118/200
183/183 [==============================] - 4s 21ms/step - loss: 2.7330 - accuracy: 0.7261 - val_loss: 2.8927 - val_accuracy: 0.5659
Epoch 119/200
183/183 [==============================] - 4s 21ms/step - loss: 2.6924 - accuracy: 0.7648 - val_loss: 2.9258 - val_accuracy:

183/183 [==============================] - 4s 21ms/step - loss: 2.6177 - accuracy: 0.8392 - val_loss: 2.8487 - val_accuracy: 0.6050
Epoch 170/200
183/183 [==============================] - 4s 21ms/step - loss: 2.6236 - accuracy: 0.8333 - val_loss: 2.8298 - val_accuracy: 0.6295
Epoch 171/200
183/183 [==============================] - 4s 21ms/step - loss: 2.6187 - accuracy: 0.8388 - val_loss: 2.8668 - val_accuracy: 0.5904
Epoch 172/200
183/183 [==============================] - 4s 21ms/step - loss: 2.6231 - accuracy: 0.8347 - val_loss: 2.8333 - val_accuracy: 0.6215
Epoch 173/200
183/183 [==============================] - 4s 21ms/step - loss: 2.6217 - accuracy: 0.8359 - val_loss: 2.8530 - val_accuracy: 0.6032
Epoch 174/200
183/183 [==============================] - 4s 21ms/step - loss: 2.6226 - accuracy: 0.8349 - val_loss: 2.8497 - val_accuracy: 0.6076
Epoch 175/200
183/183 [==============================] - 4s 21ms/step - loss: 2.6184 - accuracy: 0.8393 - val_loss: 2.8677 - val_accuracy:

In [96]:
model.save('model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model\assets


In [103]:
audio_in = sd.InputStream(samplerate = 16000, channels = 1)
audio_in.start()
data_rd, _ = audio_in.read(16000)
audio_in.stop()
print('stopped')
sd.play(data_rd, 16000)
time.sleep(1.0)
sd.stop()

stopped


In [97]:
REAL_CATEGORIES = ["bed", "bird", "cat", "dog", "down", "eight", "five", "four","go","happy","house","left","marvin","nine","no","off","on","one","right","seven","sheila","six","stop","three","tree","two","up","wow","yes","zero"]
revDict = dict()
for i in range(len(REAL_CATEGORIES)):
    revDict[i] = REAL_CATEGORIES[i]

In [104]:
data_rd = data_rd.astype(float)
data_rd = data_rd.reshape(16000)
sd.play(data_rd, 16000)
time.sleep(1.0)
sd.stop()
mfcc_sample = lb.feature.mfcc(y = data_rd, sr = 16000, n_mfcc = 32)
pred = model.predict(mfcc_sample.reshape(1,32,32,1))

print('prediction:', revDict[np.argmax(pred)])

prediction: off


In [103]:
p = Path(r'input/three/2ce7534c_nohash_0.wav')
_, data_ll = wavfile.read(p)
data_ll = data_ll.astype(float)
mfcc_sample = lb.feature.mfcc(y = data_ll, sr = 16000, n_mfcc = 32)
if(mfcc_sample.shape[1] != 32):
        temp_shell = np.zeros((32,32))
        temp_shell[0:mfcc_sample.shape[0], 0:mfcc_sample.shape[1]] += mfcc_sample
        pred = model.predict(temp_shell.reshape(1,32,32,1))
else:
    pred = model.predict(mfcc_sample.reshape(1,32,32,1))

print('prediction:', revDict[np.argmax(pred)])

prediction: three
